# Hệ thống gợi ý dựa vào nội dung (Content-based RecSys)
Hệ thống gợi ý sản phẩm cho user mục tiêu bằng cách tìm kiếm những sản phẩm tương đồng với các sản phẩm mà user đã đánh giá trong quá khứ.

Dữ liệu đầu vào là ma trận gồm các thuộc tính của sản phẩm (item content matrix-ICM), với các dòng của ma trận đại diện cho từng item, các cột là các thuộc tính của item, các giá trị của ma trận nhận giá trị 0 hoặc 1, 1 nếu item có thuộc tính đó, ngược lại là 0, và cột cuối như một biến phân loại xem user mục tiêu có thích item tương ứng hay không.

**Term Frequency (TF) and Inverse Document Frequency (IDF)**

là một kỹ thuật dùng để hiệu chỉnh trọng số giữa các thuộc tính của item, dựa vào tần suất xuất hiện của chúng. Cụ thể, nếu thuộc tính $a$ có tần suất xuất hiện nhiều thì giá trị IDF sẽ thấp, nghĩa là ta không quan tâm nhiều đến thuộc tính có mặt ở khắp mọi nơi mà ta chú ý nhiều đến những thuộc tính hiếm gặp.
$$\text{tfidf}_{i,j} = \text{tf}_{i,j} \times \log \left({\frac{N}{\text{df}_i + 1}}\right) $$
trong đó
- $\text{tf}_{i,j}$: tổng số lần xuất hiện của thuộc tính $i$ trong item $j$
- $\text{df}_i$: tổng số item chứa thuộc tính $i$
- $N$: tổng số item có trong tập dữ liệu

**TH ma trận là nhị phân:**
- Chuẩn hóa ma trận sao cho mỗi dòng có độ dài bằng 1
- Tính
$$\text{IDF}_{i} = \left\{ \begin{array}{rcl}
1 + \log_{10} \frac{1}{\text{DF}_{i}} & \text{ if DF}_{i} >0 \\
0 & \text{otherwise}
\end{array}\right.$$
- Ma trận có trọng số = ma trận được chuẩn hóa * IDF

**User profile**

Để tìm ra sở thích của user $u$ với từng thuộc tính trong ma trận ICM, sử dụng tập dữ liệu chỉ gồm các item mà user $u$ đã đánh giá trong quá khứ và số điểm mà user đó đã đánh giá.

$$\text{User profile}_a = \text{dotproduct}(\text{rating,attr}_a)$$

**Ước lượng số điểm đánh giá**

pred_rating = dotproduct(user profile,ma trận có trọng số)


## **Ví dụ**

Tập dữ liệu gồm 8 phim (dòng) và 6 thể loại phim (cột). Cột `rating` là sự yêu thích của user đối với các phim.

|Genre =>|Comedy|Drama|Romance|Thriller|Action|Horror|rating|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|1|1|0|1|0|0|0|Dislike|
|2|1|1|1|0|1|0|Dislike|
|3|1|1|0|0|0|0|Dislike|
|4|0|0|0|1|1|0|Like|
|5|0|1|0|1|1|1|Like|
|6|0|0|0|0|1|1|Like|
|Test-1|0|0|0|1|0|1|?|
|Test-2|0|1|1|0|0|0|?|

Yêu cầu: dự đoán rating của Test-1 và Test-2 (sử dụng TFIDF)

**Các bước thực hiện**
1. Chuẩn hóa ma trận
2. Tạo user profile
3. Tạo ma trận có trọng số (tfidf)
4. Dự đoán số điểm user có thể đánh giá cho các item
5. Chọn $k$ items có số điểm đánh giá dự đoán cao nhất

**Chuẩn hóa ma trận, Tạo user profile, Tính IDF**

|Genre =>|Comedy|Drama|Romance|Thriller|Action|Horror|Total features|User_rating|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|1|$\frac{1}{\sqrt{2}}$|0|$\frac{1}{\sqrt{2}}$|0|0|0|2|-1|
|2|$\frac{1}{\sqrt{4}}$|$\frac{1}{\sqrt{4}}$|$\frac{1}{\sqrt{4}}$|0|$\frac{1}{\sqrt{4}}$|0|4|-1|
|3|$\frac{1}{\sqrt{2}}$|$\frac{1}{\sqrt{2}}$|0|0|0|0|2|-1|
|4|0|0|0|$\frac{1}{\sqrt{2}}$|$\frac{1}{\sqrt{2}}$|0|2|1|
|5|0|$\frac{1}{\sqrt{4}}$|0|$\frac{1}{\sqrt{4}}$|$\frac{1}{\sqrt{4}}$|$\frac{1}{\sqrt{4}}$|4|1|
|6|0|0|0|0|$\frac{1}{\sqrt{2}}$|$\frac{1}{\sqrt{2}}$|2|1|
|Test-1|0|0|0|$\frac{1}{\sqrt{2}}$|0|$\frac{1}{\sqrt{2}}$|2|?|
|Test-2|0|$\frac{1}{\sqrt{2}}$|$\frac{1}{\sqrt{2}}$|0|0|0|2|?|
|User profile|-1.914|-0.707|-1.207|1.207|1.414|1.207|
|DF|3|4|3|3|4|3|
|IDF|0.523|0.398|0.523|0.523|0.398|0.523|

- User profile = dotproduct(user_rating,genre)
$$\text{dotproduct(user_rating,Comedy)} =  (-1)*\frac{1}{\sqrt{2}} + (-1)*\frac{1}{\sqrt{4}} + (-1)*\frac{1}{\sqrt{2}} = -1.914
  $$
- IDF = $1 + \log_{10} \frac{1}{\text{DF}}$

**Ma trận có trọng số:**

|Genre =>|Comedy|Drama|Romance|Thriller|Action|Horror|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|1|0.37|0|0.37|0|0|0|
|2|0.2615|0.199|0.2615|0|0.199|0|
|3|0.37|0.2814|0|0|0|0|
|4|0|0|0|0.37|0.2814|0|
|5|0|0.199|0|0.2615|0.199|0.2615|
|6|0|0|0|0|0.2814|0.37|
|Test-1|0|0|0|0.37|0|0.37|
|Test-2|0|0.2814|0.37|0|0|0|
|User profile|-1.914|-0.707|-1.207|1.207|1.414|1.207|

**Dự đoán rating**

pred_rating = dotproduct(user profile,ma trận có trọng số)
$$\text{Predicted rating (Test-1)} = 1.207*0.37 + 1.207*0.37 = 0.893$$

|Genre =>|Comedy|Drama|Romance|Thriller|Action|Horror|User_rating|Predicted_rating|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|1|0.37|0|0.37|0|0|0|-1|-1.154|
|2|0.2615|0.199|0.2615|0|0.199|0|-1|-0.675|
|3|0.37|0.2814|0|0|0|0|-1|-0.907|
|4|0|0|0|0.37|0.2814|0|1|0.844|
|5|0|0.199|0|0.2615|0.199|0.2615|1|0.772|
|6|0|0|0|0|0.2814|0.37|1|0.844|
|Test-1|0|0|0|0.37|0|0.37|?|0.893|
|Test-2|0|0.2814|0.37|0|0|0|?|-0.645|
|User profile|-1.914|-0.707|-1.207|1.207|1.414|1.207|


### Sử dụng Python

In [1]:
import numpy as np
import pandas as pd

In [24]:
mx = np.array([[1,0,1,0,0,0],
      [1,1,1,0,1,0],
      [1,1,0,0,0,0],
      [0,0,0,1,1,0],
      [0,1,0,1,1,1],
      [0,0,0,0,1,1],
      [0,0,0,1,0,1],
      [0,1,1,0,0,0]])
movieId = [1,2,3,4,5,6,'Test-1', 'Test-2']
columns=['Comedy','Drama','Romance','Thriller','Action','Horror']
likeornot = ['dislike', 'dislike', 'dislike', 'like', 'like', 'like', '?', '?']

example_df = pd.DataFrame(mx, columns=columns, index=movieId)
example_df['rating'] = likeornot
example_df['rating'] = example_df['rating'].apply(lambda x: -1 if x == 'dislike' else 1 if  x == 'like' else np.nan)
display(example_df)

# Tách data thành tập X và y
X = example_df.iloc[:,:-1]
y = example_df.iloc[:,-1]

,Comedy,Drama,Romance,Thriller,Action,Horror,rating
1,1,0,1,0,0,0,-1.0
2,1,1,1,0,1,0,-1.0
3,1,1,0,0,0,0,-1.0
4,0,0,0,1,1,0,1.0
5,0,1,0,1,1,1,1.0
6,0,0,0,0,1,1,1.0
Test-1,0,0,0,1,0,1,NaN
Test-2,0,1,1,0,0,0,NaN


In [25]:
def normalized_X(X):
  X['total_attrs'] = X.sum(axis=1)
  for i in X.columns:
    if i != 'total_attrs':
      X[i] = X[i]/np.sqrt(X['total_attrs'])
  X_rm = X.drop(columns=['total_attrs'], axis=0)

  return X_rm

def idf(X):
  df = X.sum(axis=0)
  idf = 1 + np.log10(1/df)

  return idf

In [26]:
idf = idf(X)
print(idf)
X_nr = normalized_X(X)
X_nr

Comedy      0.522879
Drama       0.397940
Romance     0.522879
Thriller    0.522879
Action      0.397940
Horror      0.522879
dtype: float64


,Comedy,Drama,Romance,Thriller,Action,Horror
1,0.707107,0.000000,0.707107,0.000000,0.000000,0.000000
2,0.500000,0.500000,0.500000,0.000000,0.500000,0.000000
3,0.707107,0.707107,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.707107,0.707107,0.000000
5,0.000000,0.500000,0.000000,0.500000,0.500000,0.500000
6,0.000000,0.000000,0.000000,0.000000,0.707107,0.707107
Test-1,0.000000,0.000000,0.000000,0.707107,0.000000,0.707107
Test-2,0.000000,0.707107,0.707107,0.000000,0.000000,0.000000


In [27]:
# Hàm tạo thông tin về user
def user_profile(normalized_X,y):

  # Lọc ra các index mà user đã đánh giá
  y_ind = y[~y.isna()]

  # tạo dataframe chỉ gồm những item mà user đã đánh giá
  df = pd.merge(normalized_X, y_ind, left_index=True, right_index=True, how='inner')

  # Dot product(attribute, rating)
  user_profile = [df.rating.dot(df[i]) for i in df.columns[:-1]]

  return user_profile

In [28]:
def rating_prediction(weighted_X, user_profile):
  # Sử dụng sở thích của user về các item mà user đã đánh giá để dự đoán rating
   pred_rating = weighted_X.dot(user_profile)
   return pd.DataFrame(pred_rating, columns=['pred_rating']).round(3)


In [29]:
# Dự đoán rating
result_df_3 = pd.merge(y, rating_prediction(X_nr*idf, user_profile(X_nr,y)), left_index=True, right_index=True)
display(result_df_3)

,rating,pred_rating
1,-1.0,-1.154
2,-1.0,-0.675
3,-1.0,-0.907
4,1.0,0.844
5,1.0,0.772
6,1.0,0.844
Test-1,NaN,0.893
Test-2,NaN,-0.645


## **Bài tập**

Tập dữ liệu có kích thước 20x12, gồm 20 doc (dòng), 10 thuộc tính (20 cột đầu) và 2 cột cuối là đánh giá của user1và user2.

Link file: https://docs.google.com/spreadsheets/d/14tqmXDkOS_EZM4Z_pewSxSPfBMxKTTUp/edit?usp=drive_web&ouid=115422524039268283690&rtpof=true

1. Thực hiện các yêu cầu như trong ví dụ trên cho từng user.
2. Chọn ra 3 docs (mà user chưa đánh giá) có số điểm đánh giá dự đoán cao nhất của từng user.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount("/content/drive")

# Thay đổi đương dẫn
import os
os.chdir("drive/My Drive/KHTN/RecSys/data")

# Print out the current directory
!pwd

Mounted at /content/drive
/content/drive/My Drive/KHTN/RecSys/data


In [ ]:
# Load the dataset
data = pd.read_excel('data_03.xls')
data.rename(columns={"Unnamed: 0": "doc"}, inplace=True)
data.set_index('doc', inplace = True)
data


,baseball,economics,politics,Europe,Asia,soccer,war,security,shopping,family,User 1,User 2
doc,,,,,,,,,,,,
doc1,1,0,1,0,1,1,0,0,0,1,1.0,-1.0
doc2,0,1,1,1,0,0,0,1,0,0,-1.0,1.0
doc3,0,0,0,1,1,1,0,0,0,0,NaN,NaN
doc4,0,0,1,1,0,0,1,1,0,0,NaN,1.0
doc5,0,1,0,0,0,0,0,0,1,1,NaN,NaN
doc6,1,0,0,1,0,0,0,0,0,0,1.0,NaN
doc7,0,0,0,0,0,0,0,1,0,1,NaN,NaN
doc8,0,0,1,1,0,0,1,0,0,1,NaN,NaN
doc9,0,0,0,0,0,1,0,0,1,0,NaN,NaN
